In [1]:
%load_ext autoreload
%autoreload 2

# Setup

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from deepexperiment.utils import one_hot_encoding, one_hot_encoding_batch, ResBlock, get_indices
from deepexperiment.interpret import DeepShap

2022-10-30 11:17:02.362956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/katarina/Documents/repositories/GitHub/DeepExperiment/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading model and data

In [3]:
model = keras.models.load_model("../models/resnet_small_committee.h5")

2022-10-30 11:17:22.157109: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
samples = pd.read_csv('../data/positive_set_extended.csv')
print(len(samples))

18513


In [5]:
data, _ = one_hot_encoding_batch(samples)

# DeepSHAP

In [7]:
len(data) // 1000

18

In [10]:
rand_samples = samples.sample(n=100, replace=False, random_state=42).reset_index(drop=True)
background, _ = one_hot_encoding_batch(rand_samples)

for i in range((len(data) // 1000) + 1):
    filename = f'../data/positive_set_shaps_{i}.npy'
    try:
        shaps = np.load(filename)
    except:
        deepShap = DeepShap(model, background)
        data_sample = data[i*1000:(i+1)*1000]
        attribution = deepShap(data_sample)
        pos_shaps = attribution[1][:,:,:,0]
        shaps = np.array(pos_shaps)
        np.save(filename, shaps)

KeyboardInterrupt: 